In [2]:
import numpy as np
import pandas as pd

In [4]:
df = pd.read_csv(r'C:\Users\Vic\Documents\Studia\SN\Program_mentorski\RF_shap\data2.csv')
df.head()

,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
0,1,0.370594,0.424389,0.405750,0.601457,0.601457,0.998969,0.796887,0.808809,0.302646,...,0.716845,0.009219,0.622879,0.601453,0.827890,0.290202,0.026601,0.564050,1,0.016469
1,1,0.464291,0.538214,0.516730,0.610235,0.610235,0.998946,0.797380,0.809301,0.303556,...,0.795297,0.008323,0.623652,0.610237,0.839969,0.283846,0.264577,0.570175,1,0.020794
2,1,0.426071,0.499019,0.472295,0.601450,0.601364,0.998857,0.796403,0.808388,0.302035,...,0.774670,0.040003,0.623841,0.601449,0.836774,0.290189,0.026555,0.563706,1,0.016474
3,1,0.399844,0.451265,0.457733,0.583541,0.583541,0.998700,0.796967,0.808966,0.303350,...,0.739555,0.003252,0.622929,0.583538,0.834697,0.281721,0.026697,0.564663,1,0.023982
4,1,0.465022,0.538432,0.522298,0.598783,0.598783,0.998973,0.797366,0.809304,0.303475,...,0.795016,0.003878,0.623521,0.598782,0.839973,0.278514,0.024752,0.575617,1,0.035490


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

#funkcja tworząca model tylko z jedną cechą dla każdej cechy
def one_feature(label_column, df):
    df = df
    label_column = label_column
    results = {} 
    feature_columns = [col for col in df.columns if col != label_column]
   
    for feature in feature_columns:
        # Wybierz tylko jedną cechę + etykietę
        df_small = df[[feature, label_column]].dropna()
        
        # Shuffle i reset
        df_shuffled = df_small.sample(frac=1, random_state=42).reset_index(drop=True)
        
        # Podział 80/20
        train_size = int(0.8 * len(df_shuffled))
        df_train = df_shuffled.iloc[:train_size]
        df_test = df_shuffled.iloc[train_size:]
        
        # X i y
        X_train = df_train[[feature]].values
        y_train = df_train[label_column].values
        
        X_test = df_test[[feature]].values
        y_test = df_test[label_column].values
        
        # RandomForest z sklearn
        clf = RandomForestClassifier(n_estimators=20, random_state=42)
        clf.fit(X_train, y_train)
        predictions = clf.predict(X_test)
        
        # Accuracy
        acc = accuracy_score(y_test, predictions)
        results[feature] = acc  # Dodanie do słownika
    
    return results


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def plot_feature_accuracies_seaborn(results_dict):
    # Słownik do DataFrame
    df_results = pd.DataFrame(list(results_dict.items()), columns=["Feature", "Accuracy"])
    
    # Sortowanie według dokładności
    df_results = df_results.sort_values("Accuracy", ascending=False)

    # Wykres
    plt.figure(figsize=(12, 6))
    sns.barplot(data=df_results, x="Accuracy", y="Feature", palette="viridis")
    plt.title("Dokładność RandomForest dla pojedynczych cech")
    plt.xlabel("Dokładność")
    plt.ylabel("Cechy")
    plt.xlim(0, 1)  # Oś dokładności od 0 do 1
    plt.tight_layout()
    plt.show()


In [65]:
results = one_feature('Bankrupt?', df)
sorted_results = sorted(results.items(), key=lambda x: x[1], reverse=True)

for i, (feature, acc) in enumerate(sorted_results, 1):
    print(f"{i}. Feature: {feature}, Accuracy: {acc:.4f}")


1. Feature:  Persistent EPS in the Last Four Seasons, Accuracy: 0.9648
2. Feature:  Total Asset Turnover, Accuracy: 0.9641
3. Feature:  Net Income Flag, Accuracy: 0.9641
4. Feature:  Liability-Assets Flag, Accuracy: 0.9633
5. Feature:  Net Worth Turnover Rate (times), Accuracy: 0.9626
6. Feature:  Cash Flow Per Share, Accuracy: 0.9582
7. Feature:  Operating Profit Per Share (Yuan ¥), Accuracy: 0.9582
8. Feature:  Total Asset Growth Rate, Accuracy: 0.9582
9. Feature:  Tax rate (A), Accuracy: 0.9575
10. Feature:  Inventory Turnover Rate (times), Accuracy: 0.9575
11. Feature:  Interest-bearing debt interest rate, Accuracy: 0.9567
12. Feature:  Net Value Per Share (B), Accuracy: 0.9567
13. Feature:  Net Value Per Share (A), Accuracy: 0.9567
14. Feature:  Research and development expense rate, Accuracy: 0.9560
15. Feature:  Contingent liabilities/Net worth, Accuracy: 0.9560
16. Feature:  Net Value Per Share (C), Accuracy: 0.9553
17. Feature:  After-tax net Interest Rate, Accuracy: 0.9545
18

In [33]:
df2 = pd.read_csv(r'C:\Users\Vic\Documents\Studia\SN\Program_mentorski\RF_shap\manufacturing_defect_dataset.csv')
df2.head()

,ProductionVolume,ProductionCost,SupplierQuality,DeliveryDelay,DefectRate,QualityScore,MaintenanceHours,DowntimePercentage,InventoryTurnover,StockoutRate,WorkerProductivity,SafetyIncidents,EnergyConsumption,EnergyEfficiency,AdditiveProcessTime,AdditiveMaterialCost,DefectStatus
0,202,13175.403783,86.648534,1,3.121492,63.463494,9,0.052343,8.630515,0.081322,85.042379,0,2419.616785,0.468947,5.551639,236.439301,1
1,535,19770.046093,86.310664,4,0.819531,83.697818,20,4.908328,9.296598,0.038486,99.657443,7,3915.566713,0.119485,9.080754,353.957631,1
2,960,19060.820997,82.132472,0,4.514504,90.350550,1,2.464923,5.097486,0.002887,92.819264,2,3392.385362,0.496392,6.562827,396.189402,1
3,370,5647.606037,87.335966,5,0.638524,67.628690,8,4.692476,3.577616,0.055331,96.887013,8,4652.400275,0.183125,8.097496,164.135870,1
4,206,7472.222236,81.989893,3,3.867784,82.728334,9,2.746726,6.851709,0.068047,88.315554,7,1581.630332,0.263507,6.406154,365.708964,1


In [35]:
one_feature('DefectStatus', df2)

1. Feature: ProductionVolume, Accuracy: 0.7623
2. Feature: ProductionCost, Accuracy: 0.7377
3. Feature: SupplierQuality, Accuracy: 0.7299
4. Feature: DeliveryDelay, Accuracy: 0.8287
5. Feature: DefectRate, Accuracy: 0.7485
6. Feature: QualityScore, Accuracy: 0.7083
7. Feature: MaintenanceHours, Accuracy: 0.8287
8. Feature: DowntimePercentage, Accuracy: 0.7114
9. Feature: InventoryTurnover, Accuracy: 0.7608
10. Feature: StockoutRate, Accuracy: 0.7207
11. Feature: WorkerProductivity, Accuracy: 0.7361
12. Feature: SafetyIncidents, Accuracy: 0.8287
13. Feature: EnergyConsumption, Accuracy: 0.7253
14. Feature: EnergyEfficiency, Accuracy: 0.7299
15. Feature: AdditiveProcessTime, Accuracy: 0.7299
16. Feature: AdditiveMaterialCost, Accuracy: 0.7022


1

In [67]:
df3 = pd.read_csv(r'C:\Users\Vic\Documents\Studia\SN\Program_mentorski\RF_shap\data.csv')
df3.head(30)

,footfall,tempMode,AQ,USS,CS,VOC,RP,IP,Temperature,fail
0,0,7,7,1,6,6,36,3,1,1
1,190,1,3,3,5,1,20,4,1,0
2,31,7,2,2,6,1,24,6,1,0
3,83,4,3,4,5,1,28,6,1,0
4,640,7,5,6,4,0,68,6,1,0
5,110,3,3,4,6,1,21,4,1,0
6,100,7,5,6,4,1,77,4,1,0
7,31,1,5,4,5,4,21,4,1,0
8,180,7,4,6,3,3,31,4,1,0
9,2800,0,3,3,7,0,39,3,1,0


In [39]:
one_feature('fail', df3)

1. Feature: footfall, Accuracy: 0.5820
2. Feature: tempMode, Accuracy: 0.6402
3. Feature: AQ, Accuracy: 0.7513
4. Feature: USS, Accuracy: 0.7407
5. Feature: CS, Accuracy: 0.6614
6. Feature: VOC, Accuracy: 0.9153
7. Feature: RP, Accuracy: 0.5661
8. Feature: IP, Accuracy: 0.6349
9. Feature: Temperature, Accuracy: 0.5979


1

In [43]:
df4 = pd.read_csv(r'C:\Users\Vic\Documents\Studia\SN\Program_mentorski\RF_shap\Employee.csv')
df4.head()

,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot
0,Bachelors,2017,Bangalore,3,34,Male,No,0,0
1,Bachelors,2013,Pune,1,28,Female,No,3,1
2,Bachelors,2014,New Delhi,3,38,Female,No,2,0
3,Masters,2016,Bangalore,3,27,Male,No,5,1
4,Masters,2017,Pune,3,24,Male,Yes,2,1


In [47]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4653 entries, 0 to 4652
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Education                  4653 non-null   object
 1   JoiningYear                4653 non-null   int64 
 2   City                       4653 non-null   object
 3   PaymentTier                4653 non-null   int64 
 4   Age                        4653 non-null   int64 
 5   Gender                     4653 non-null   object
 6   EverBenched                4653 non-null   object
 7   ExperienceInCurrentDomain  4653 non-null   int64 
 8   LeaveOrNot                 4653 non-null   int64 
dtypes: int64(5), object(4)
memory usage: 327.3+ KB


In [49]:
# Zamiana tekstów na liczby
df4['Gender'] = pd.factorize(df4['Gender'])[0]
df4['Education'] = pd.factorize(df4['Education'])[0]
df4['City'] = pd.factorize(df4['City'])[0]

In [51]:
df4['EverBenched'] = pd.factorize(df4['EverBenched'])[0]

In [53]:
one_feature('LeaveOrNot', df4)

1. Feature: Education, Accuracy: 0.6649
2. Feature: JoiningYear, Accuracy: 0.7433
3. Feature: City, Accuracy: 0.6617
4. Feature: PaymentTier, Accuracy: 0.6821
5. Feature: Age, Accuracy: 0.6649
6. Feature: Gender, Accuracy: 0.6649
7. Feature: EverBenched, Accuracy: 0.6649
8. Feature: ExperienceInCurrentDomain, Accuracy: 0.6617


1